In [ ]:
import os, json

from llama_index.core.workflow import (
    step,
    Context,
    Workflow,
    Event,
    StartEvent,
    StopEvent,
)

from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows

import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
from llama_index.tools.tavily_research.base import TavilyToolSpec
from llama_index.core import PromptTemplate
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import ReActAgent

import llama_index
import importlib.metadata
print(importlib.metadata.version("llama-index-core"))

# exam

In [ ]:
qset = {
  "id": "113-1-1-med-surg",
  "year": "113",
  "time": "1",
  "qid": "1",
  "discipline": "內外科護理學",
  "ans": "C",
  "question": "有關多發性硬化症之診斷檢查，下列何者錯誤？",
  "options": {
   "A": "腦脊髓液分析可發現IgG抗體上升",
   "B": "視覺誘發電位可觀察到受損的神經在傳導過程出現延遲和中斷",
   "C": "超音波檢查可發現中樞神經系統髓鞘脫失",
   "D": "核磁共振影像可用來確認多發性硬化症之斑塊"
  },
  "discipline_slug": "med-surg"
}

exam_question = f"題幹: {qset['question']}\n選項: \nA: {qset['options']['A']}; B: {qset['options']['B']}; C: {qset['options']['C']}; D: {qset['options']['D']}."
print(exam_question)

# SubQuestions

In [ ]:
sub_question_prompt = PromptTemplate("""你是一個考題拆解助手。請根據以下單選題，產生一系列子問題。規則如下：

1. 子問題需為單一問句，避免複合句。
2. 每個子問題必須包含完整上下文，不可依賴原始題目才能理解。
3. 子問題的集合在合併答案後，應能完整回答此單選題。
4. 回應必須是**純 JSON 格式**，不得包含任何額外文字或 Markdown。

### 範例輸出：
{{
  "sub_questions": [
    "舊金山的人口是多少？",
    "舊金山的年度預算是多少？",
    "舊金山的 GDP 是多少？"
  ]
}}

以下是單選題：
{exam_question}
"""
)

In [ ]:
print(sub_question_prompt.format(exam_question = exam_question))

In [ ]:
llm = OpenAI(
    model="gpt-5-mini",
    temperature=0,
    json_mode=True,
)
response = llm.complete(sub_question_prompt.format(exam_question=exam_question))
json.loads(response.text)

In [ ]:
sub_questions = json.loads(response.text)['sub_questions']
sub_questions

# ReActAgent

### build tools

In [ ]:
tavily_tool = TavilyToolSpec(
    api_key=TAVILY_API_KEY,
)
tavily_tool_list = tavily_tool.to_tool_list()
type(tavily_tool_list[0])  # llama_index.core.tools.function_tool.FunctionTool
llm = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False)  # streaming False for non-verified organisations
agent = ReActAgent(tools=tavily_tool_list, llm=llm, streaming=False, verbose=False)

### ReActAgent Run

In [ ]:
from llama_index.core.agent.workflow import AgentInput, AgentOutput, ToolCall, ToolCallResult

async def run_agent_with_stream(agent, query):
    handler = agent.run(query)
    results = []

    async for ev in handler.stream_events(expose_internal=False):
        name = ev.__class__.__name__
        print(f"-----stream event: {name}")
        results.append((name, ev))

        if isinstance(ev, AgentInput):
            print(f"len of chat message: {len(ev.input)}")
        elif isinstance(ev, AgentOutput):
            print(ev.response.blocks[0].text)
        elif isinstance(ev, ToolCall):
            print(f"{ev.tool_name}: {ev.tool_kwargs}")
        elif isinstance(ev, ToolCallResult):
            num_rv = len(ev.tool_output.blocks)
            print(f"num_result: {num_rv}")

    # 最終 response
    response = await handler
    return results, response

results, response = await run_agent_with_stream(agent, sub_questions[0])
# await agent.run(query, ctx=ctx)
#response = await agent.run(sub_questions[0])

### Customizing the Prompt

In [ ]:
print(agent.get_prompts()['react_header'].template)

In [ ]:
AGENT_PROMPT = PromptTemplate("""你被設計來協助處理各種任務，包含提供查詢到的資料用以回答問題、提供摘要以及其他類型的分析。
並且為了確保資訊的更新，請盡可能的在有相關查詢內容的情況下才回答。

## 工具

你可以使用各式各樣的工具。你需要自行決定使用這些工具的順序來完成任務。
這可能需要將任務拆解成子任務，並使用不同的工具來完成每個子任務。

你能使用以下工具：
{tool_desc}


## 輸出格式

請用與問題相同的語言回答，並使用以下格式：

```

Thought: 我需要使用一個工具來協助回答問題。
Action: 工具名稱 (必須是 {tool_names} 之一，如果要用工具的話)
Action Input: 工具的輸入，必須是 JSON 格式，代表 kwargs (例如 {{"input": "hello world", "num_beams": 5}})

```

請務必**總是以 Thought 開始**。

絕對不要用 Markdown 的程式碼區塊把整個回應包起來。但如果需要，你可以在回應中使用程式碼區塊。

請務必使用合法的 JSON 格式作為 Action Input。
如果你包含了 "Action:" 這一行，那麼你也必須包含 "Action Input:" 這一行，即使該工具不需要任何參數，此時你也必須寫 "Action Input: {{}}"。

如果使用這個格式，工具會回傳以下格式的結果：

```

Observation: 工具的回應

```

你需要重複上述格式，直到你有足夠的資訊來回答問題而不再需要使用工具。  
這時你必須用以下兩種格式之一來回答：

```

Thought: 我已經可以不用更多工具就回答問題。
Answer: [在這裡寫下你的答案]

```
```

Thought: 我無法使用提供的工具回答問題。
Answer: [在這裡寫下你的答案]

```

## 當前對話

以下是目前的對話內容，由使用者與助手交錯的訊息組成：
"""
)

In [ ]:
print(AGENT_PROMPT.template_vars)
agent.update_prompts({"react_header": AGENT_PROMPT})

In [ ]:
results, response = await run_agent_with_stream(agent, sub_questions[0])

# combine_answers

In [ ]:
qas = []
for question in sub_questions:
    #response = await agent.run(question)
    print(f"##### question: {question}")
    results, response = await run_agent_with_stream(agent, question)
    answer = response.response.blocks[0].text
    qas.append({
        'question': question,
        'answer': answer,
    })

In [ ]:
sub_qa = "\n\n".join([
    f"Question: {qa['question']}: \n Answer: {qa['answer']}"
    for qa in qas
])
print(sub_qa)

# combine answer

In [ ]:
COMBINE_ANSWER_PROMPT = PromptTemplate("""你是一個考題作答助手。以下是一題單選題，已經被拆解成數個子問題，並且每個子問題都已有答案。
請將所有子問題的答案整合，產生一個完整且連貫的最終解答，以回答原始單選題。

以下是單選題：
{exam_question}

子問題與答案：
{sub_qa}
""")
print(COMBINE_ANSWER_PROMPT.format(sub_qa=sub_qa, exam_question=exam_question))

In [ ]:
response = llm.complete(COMBINE_ANSWER_PROMPT.format(sub_qa=sub_qa, exam_question=exam_question))
print(response)

In [ ]:
response.text